In [1]:
# "magic" commands, prefaced with "%", changes settings in the notebook

# this ensures plots are embedded in notebook web page
%matplotlib inline

# pdb = Python debugger, so this command turns the debugger OFF
%pdb off

# numpy = numerical Python, implements arrays (/ matrices)
import numpy as np
# limit number of decimal places printed for floating-point numbers
np.set_printoptions(precision=3)

# scipy = scientific Python, implements operations on arrays / matrices
import scipy as sp
# linalg = linear algebra, implements eigenvalues, matrix inverse, etc
from scipy import linalg as la
# optimize = optimization, root finding, etc
from scipy import optimize as op

# produce matlab-style plots
import matplotlib as mpl
# increase font size on plots
mpl.rc('font',**{'size':18})
# use LaTeX to render symbols
mpl.rc('text',usetex=False)
# animation
from matplotlib import animation as ani
# Matlab-style plotting
import matplotlib.pyplot as plt

# symbolic computation, i.e. computer algebra (like Mathematica, Wolfram Alpha)
import sympy as sym

Automatic pdb calling has been turned OFF


## Game Theory Notebook 

Defines the potential cost function, linearizes around the stationary points, and finds the eigenvalues of the Jacobian of the dynamics
 NOTE: this code works best in Google Colab
 

In [ ]:
# test whether this is a Colaboratory or Jupyter notebook
try:
  import google.colab
  COLAB = True
  print('Colaboratory Notebook')
except:
  COLAB = False
  print('Jupyter Notebook')

# Colab notebook
if COLAB:
  # render SymPy equations nicely in Colaboratory Notebook
  def colab_latex_printer(exp,**options):
    from google.colab.output._publish import javascript
    url = "https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.3/latest.js?config=default"
    javascript(url=url)
    return sym.printing.latex(exp,**options)

  sym.init_printing(use_latex="mathjax",latex_printer=colab_latex_printer)

# Jupyter notebook
else:
  init_printing(use_latex='mathjax')

Colaboratory Notebook


In [ ]:
import sympy as sym
from sympy import symbols, diff, Matrix, solve

In [ ]:
k, w, y, tau, gam, gam_k, gam_w, lam_k, lam_w, lam, alpha, f, a, b = symbols(r'k w y \tau \gamma \gamma_k \gamma_w \lambda_k \lambda_w \lambda \alpha f a b')
k,w,y, tau,gam,gam_k,gam_w, lam_k, lam_w, lam, alpha, f, a, b

(k, w, y, \tau, \gamma, \gammaₖ, \gamma_w, \lambdaₖ, \lambda_w, \lambda, \alph
a, f, a, b)

In [ ]:
lam, lam_k, lam_w = sym.symbols(r'\lambda \lambda_k \lambda_w',nonnegative=True)

In [ ]:
# e_scl =  (((tau**2)*(1 - k*w)**2) + (sig_w * w**2) + (sig_k*k**2)) / 2

# original scalar case
# e_scl =   (tau - (k*(w*tau)))**2 + (lam_w * w**2) + (lam_k*k**2)

# testing with velocity case
e_scl =   ((tau) - (k*(w*(y))))**2 + (lam_w * w**2) + (lam_k*k**2)


# e_scl =   (tau - (k*(w*tau + b)))**2 + (sig_w * w**2) + (sig_k*k**2)
# sig_w and sig_k are penalty parameters
# 1/2 term added to make the derivative simpler

In [ ]:
e_scl

          2              2                 2
\lambdaₖ⋅k  + \lambda_w⋅w  + (\tau - k⋅w⋅y) 

In [ ]:
# First Derivative
de_dk = diff(e_scl,k)
de_dw = diff(e_scl, w)

# Second Derivative
de2_d2k = diff(de_dk, k)
de2_dwdk = diff(de_dk, w)
de2_d2w = diff(de_dw, w)
de2_dkdw = diff(de_dw, k)

In [ ]:
de2 = [[de2_d2k, de2_dwdk],[de2_dkdw, de2_d2w]]
de2


⎡⎡                2  2         2                     ⎤  ⎡       2             
⎣⎣2⋅\lambdaₖ + 2⋅w ⋅y , 2⋅k⋅w⋅y  - 2⋅y⋅(\tau - k⋅w⋅y)⎦, ⎣2⋅k⋅w⋅y  - 2⋅y⋅(\tau 

                           2  2⎤⎤
- k⋅w⋅y), 2⋅\lambda_w + 2⋅k ⋅y ⎦⎦

In [ ]:
de = [de_dk, de_dw]
de
# non-linear dynamics -- b/c of w^2 and k^2
# w/k are evolving continuously in time based on gradient

[2⋅\lambdaₖ⋅k - 2⋅w⋅y⋅(\tau - k⋅w⋅y), 2⋅\lambda_w⋅w - 2⋅k⋅y⋅(\tau - k⋅w⋅y)]

Continuous-time gradient descent has dynamics:

$$ \dot{x} = - Dp(x) = f(x), $$

So the Jacobian of the dynamics is determined by the Hessian of error $e$:

$$ Df(x) = - D^2 p(x). $$

At a minimum $x_0$, the eigenvalues of $D^2 e(x_0)$ are positive, so the eigenvalues of $Df(x_0)$ are negative.

Note: The Hessian matrix of a function f is the Jacobian matrix of the gradient of the function: H(f(x)) = J(∇f(x)). (https://en.wikipedia.org/wiki/Hessian_matrix)



In [ ]:
Gamma = sym.Matrix.diag([1,1])
Gamma

⎡1  0⎤
⎢    ⎥
⎣0  1⎦

In [ ]:
# sets f(x) = -De(x)
f = -Gamma*sym.Matrix(de)
f

⎡-2⋅\lambdaₖ⋅k + 2⋅w⋅y⋅(\tau - k⋅w⋅y) ⎤
⎢                                     ⎥
⎣-2⋅\lambda_w⋅w + 2⋅k⋅y⋅(\tau - k⋅w⋅y)⎦

In [ ]:
# substitutes tau = 1, k_sig = w_sig = sig

subs = {lam_k:lam, lam_w:lam}

Step 1: Solving for fixed points ($\bar{x}$ such that $\frac{d}{dt}$ evaluated at $\bar{x}$ = 0, or x so that the first derivative of $\frac{df(x)}{dt}$ = 0)

In [ ]:
# stationary points
# these stationary points lead to points where e(k_0, w_0) is a minimum, maximum or saddle point
# Note: without the penalty terms in the cost function, (0,0) becomes a saddle point as a stationary point
sol = solve(de,[k,w]) # <-- find [k_0,w_0] that makes de(k_0,w_0) == [0,0]
sol

⎡        ⎛                                                                  __
⎢        ⎜           ⎛              ⎛    __________                ⎞⎞      ╱  
⎢        ⎜           ⎜            2 ⎜  ╲╱ \lambdaₖ ⋅\tau   \lambdaₖ⎟⎟     ╱   
⎢        ⎜-\lambda_w⋅⎜\lambdaₖ + y ⋅⎜- ───────────────── - ────────⎟⎟⋅   ╱   -
⎢        ⎜           ⎜              ⎜     ___________          2   ⎟⎟   ╱     
⎢        ⎜           ⎝              ⎝   ╲╱ \lambda_w ⋅y       y    ⎠⎠ ╲╱      
⎢(0, 0), ⎜────────────────────────────────────────────────────────────────────
⎢        ⎜                                          \lambdaₖ⋅\tau⋅y           
⎣        ⎝                                                                    

______________________________                                          ⎞  ⎛  
   __________                                                           ⎟  ⎜  
 ╲╱ \lambdaₖ ⋅\tau   \lambdaₖ                                           ⎟  ⎜  
 ───────────────── - ────────           ___________


Here we're just looking at particular substitutions of the fixed point


In [ ]:
# substitutes subs for the fixed points
# 2 are imaginary -- ignored
# other 2 are real if the sigma < 1 -- sigma can't be too large
# if sigma is too large, going back to (0, 0)
[sym.simplify(sym.Matrix(_).subs(subs)) for _ in sol]

⎡     ⎡       ___________________  ⎤  ⎡      ______________________⎤  ⎡      _
⎢     ⎢      ╱ -\lambda - \tau⋅y   ⎥  ⎢     ╱ -(\lambda + \tau⋅y)  ⎥  ⎢     ╱ 
⎢     ⎢     ╱  ─────────────────   ⎥  ⎢-   ╱  ──────────────────── ⎥  ⎢-   ╱  
⎢     ⎢    ╱            2          ⎥  ⎢   ╱             2          ⎥  ⎢   ╱   
⎢⎡0⎤  ⎢  ╲╱            y           ⎥  ⎢ ╲╱             y           ⎥  ⎢ ╲╱    
⎢⎢ ⎥, ⎢                            ⎥, ⎢                            ⎥, ⎢       
⎢⎣0⎦  ⎢      ______________________⎥  ⎢       ___________________  ⎥  ⎢      _
⎢     ⎢     ╱ -(\lambda + \tau⋅y)  ⎥  ⎢      ╱ -\lambda - \tau⋅y   ⎥  ⎢     ╱ 
⎢     ⎢-   ╱  ──────────────────── ⎥  ⎢     ╱  ─────────────────   ⎥  ⎢-   ╱  
⎢     ⎢   ╱             2          ⎥  ⎢    ╱            2          ⎥  ⎢   ╱   
⎣     ⎣ ╲╱             y           ⎦  ⎣  ╲╱            y           ⎦  ⎣ ╲╱    

_____________________⎤  ⎡     ___________________⎤⎤
-(\lambda - \tau⋅y)  ⎥  ⎢    ╱ -\lambda + \tau⋅y ⎥⎥
──────────────────── ⎥  ⎢ 

In [ ]:
# if lambda = 0
subs_sim = {lam:sym.Rational(0), a: 0}
[sym.simplify(sym.Matrix(_).subs(subs).subs(subs_sim)) for _ in sol]

⎡     ⎡    ________ ⎤  ⎡     ________⎤  ⎡     ______⎤  ⎡    ______⎤⎤
⎢     ⎢   ╱ -\tau   ⎥  ⎢    ╱ -\tau  ⎥  ⎢    ╱ \tau ⎥  ⎢   ╱ \tau ⎥⎥
⎢     ⎢  ╱  ──────  ⎥  ⎢-  ╱  ────── ⎥  ⎢-  ╱  ──── ⎥  ⎢  ╱  ──── ⎥⎥
⎢⎡0⎤  ⎢╲╱     y     ⎥  ⎢ ╲╱     y    ⎥  ⎢ ╲╱    y   ⎥  ⎢╲╱    y   ⎥⎥
⎢⎢ ⎥, ⎢             ⎥, ⎢             ⎥, ⎢           ⎥, ⎢          ⎥⎥
⎢⎣0⎦  ⎢     ________⎥  ⎢    ________ ⎥  ⎢     ______⎥  ⎢    ______⎥⎥
⎢     ⎢    ╱ -\tau  ⎥  ⎢   ╱ -\tau   ⎥  ⎢    ╱ \tau ⎥  ⎢   ╱ \tau ⎥⎥
⎢     ⎢-  ╱  ────── ⎥  ⎢  ╱  ──────  ⎥  ⎢-  ╱  ──── ⎥  ⎢  ╱  ──── ⎥⎥
⎣     ⎣ ╲╱     y    ⎦  ⎣╲╱     y     ⎦  ⎣ ╲╱    y   ⎦  ⎣╲╱    y   ⎦⎦

In [ ]:
# Separating the substitutions -- this is if tau = 1 only
[sym.simplify(sym.expand(sym.simplify(sym.Matrix(_).subs({tau:1, lam_k:lam, lam_w:lam})))) for _ in sol]

⎡     ⎡       ______________  ⎤  ⎡      _________________⎤  ⎡      ___________
⎢     ⎢      ╱ -\lambda - y   ⎥  ⎢     ╱ -(\lambda + y)  ⎥  ⎢     ╱ -(\lambda 
⎢     ⎢     ╱  ────────────   ⎥  ⎢-   ╱  ─────────────── ⎥  ⎢-   ╱  ──────────
⎢     ⎢    ╱         2        ⎥  ⎢   ╱           2       ⎥  ⎢   ╱           2 
⎢⎡0⎤  ⎢  ╲╱         y         ⎥  ⎢ ╲╱           y        ⎥  ⎢ ╲╱           y  
⎢⎢ ⎥, ⎢                       ⎥, ⎢                       ⎥, ⎢                 
⎢⎣0⎦  ⎢      _________________⎥  ⎢       ______________  ⎥  ⎢      ___________
⎢     ⎢     ╱ -(\lambda + y)  ⎥  ⎢      ╱ -\lambda - y   ⎥  ⎢     ╱ -(\lambda 
⎢     ⎢-   ╱  ─────────────── ⎥  ⎢     ╱  ────────────   ⎥  ⎢-   ╱  ──────────
⎢     ⎢   ╱           2       ⎥  ⎢    ╱         2        ⎥  ⎢   ╱           2 
⎣     ⎣ ╲╱           y        ⎦  ⎣  ╲╱         y         ⎦  ⎣ ╲╱           y  

______⎤  ⎡     ______________⎤⎤
- y)  ⎥  ⎢    ╱ -\lambda + y ⎥⎥
───── ⎥  ⎢   ╱  ──────────── ⎥⎥
      ⎥  ⎢  ╱         2      ⎥⎥
  

In [ ]:
[sym.simplify(sym.expand(sym.simplify(sym.Matrix(_).subs({y:1})))) for _ in sol]

⎡     ⎡                 ______________________________________________⎤  ⎡    
⎢     ⎢4 ___________   ╱     __________                   ___________ ⎥  ⎢ 4 _
⎢     ⎢╲╱ \lambda_w ⋅╲╱  - ╲╱ \lambdaₖ ⋅\tau - \lambdaₖ⋅╲╱ \lambda_w  ⎥  ⎢-╲╱ 
⎢     ⎢───────────────────────────────────────────────────────────────⎥  ⎢────
⎢     ⎢                            __________                         ⎥  ⎢    
⎢⎡0⎤  ⎢                          ╲╱ \lambdaₖ                          ⎥  ⎢    
⎢⎢ ⎥, ⎢                                                               ⎥, ⎢    
⎢⎣0⎦  ⎢                   ________________________________            ⎥  ⎢    
⎢     ⎢                  ╱     __________                             ⎥  ⎢    
⎢     ⎢                 ╱    ╲╱ \lambdaₖ ⋅\tau                        ⎥  ⎢    
⎢     ⎢            -   ╱   - ───────────────── - \lambdaₖ             ⎥  ⎢    
⎢     ⎢               ╱          ___________                          ⎥  ⎢    
⎣     ⎣             ╲╱         ╲╱ \lambda_w         

In [ ]:
# expands de with the substitutions above
sym.expand(sym.simplify(sym.Matrix(de).subs(subs)))

⎡                                2  2⎤
⎢2⋅\lambda⋅k - 2⋅\tau⋅w⋅y + 2⋅k⋅w ⋅y ⎥
⎢                                    ⎥
⎢                              2    2⎥
⎣2⋅\lambda⋅w - 2⋅\tau⋅k⋅y + 2⋅k ⋅w⋅y ⎦

In [ ]:
k0,w0 = sym.simplify(sym.Matrix(sol[4]))
x0 = {k:k0,w:w0}
x0

⎧                       ______________________________________________        
⎪                      ╱   __________                     ___________         
⎪   4 ___________     ╱  ╲╱ \lambdaₖ ⋅\tau⋅y - \lambdaₖ⋅╲╱ \lambda_w          
⎪   ╲╱ \lambda_w ⋅   ╱   ────────────────────────────────────────────         
⎨                   ╱                          2                              
⎪                 ╲╱                          y                               
⎪k: ──────────────────────────────────────────────────────────────────, w:   ╱
⎪                                __________                                ╲╱ 
⎩                              ╲╱ \lambdaₖ                                    

     ______________________________⎫
    ╱   __________                 ⎪
   ╱  ╲╱ \lambdaₖ ⋅\tau   \lambdaₖ ⎪
  ╱   ───────────────── - ──────── ⎪
 ╱        ___________        y     ⎬
╱       ╲╱ \lambda_w               ⎪
      ──────────────────────────── ⎪
                   y         

In [ ]:
# Looking at one of the solved fixed points
k0,w0 = sym.simplify(sym.Matrix(sol[0]).subs(subs).subs({a:0}))
x0 = {k:k0,w:w0}
x0
# as sigma --> tau^s, goes to 0
# as sigma --> tau^2, all 3 equilibria coincide

{k: 0, w: 0}

In [ ]:
k0_s,w0_s = sym.simplify(sym.Matrix(sol[0]).subs(subs).subs({lam:sym.Rational(1,2), tau:sym.Rational(1), a: 0}))
x0_s = {k:k0_s,w:w0_s}
x0_s

{k: 0, w: 0}

Step 2: Find the Jacobian evaluated at the fixed points


In [ ]:
H = sym.Matrix(de).jacobian([k,w])
J = f.jacobian([k,w])

In [ ]:
f

⎡-2⋅\lambdaₖ⋅k + 2⋅w⋅y⋅(\tau - k⋅w⋅y) ⎤
⎢                                     ⎥
⎣-2⋅\lambda_w⋅w + 2⋅k⋅y⋅(\tau - k⋅w⋅y)⎦

In [ ]:
J

⎡                      2  2                2                     ⎤
⎢     -2⋅\lambdaₖ - 2⋅w ⋅y        - 2⋅k⋅w⋅y  + 2⋅y⋅(\tau - k⋅w⋅y)⎥
⎢                                                                ⎥
⎢         2                                             2  2     ⎥
⎣- 2⋅k⋅w⋅y  + 2⋅y⋅(\tau - k⋅w⋅y)      -2⋅\lambda_w - 2⋅k ⋅y      ⎦

In [ ]:
de2

⎡⎡                2  2         2                     ⎤  ⎡       2             
⎣⎣2⋅\lambdaₖ + 2⋅w ⋅y , 2⋅k⋅w⋅y  - 2⋅y⋅(\tau - k⋅w⋅y)⎦, ⎣2⋅k⋅w⋅y  - 2⋅y⋅(\tau 

                           2  2⎤⎤
- k⋅w⋅y), 2⋅\lambda_w + 2⋅k ⋅y ⎦⎦

In [ ]:
J.T

⎡                      2  2                2                     ⎤
⎢     -2⋅\lambdaₖ - 2⋅w ⋅y        - 2⋅k⋅w⋅y  + 2⋅y⋅(\tau - k⋅w⋅y)⎥
⎢                                                                ⎥
⎢         2                                             2  2     ⎥
⎣- 2⋅k⋅w⋅y  + 2⋅y⋅(\tau - k⋅w⋅y)      -2⋅\lambda_w - 2⋅k ⋅y      ⎦

In [ ]:
# Is this symmetric?
# Symmetric = potential game
# symmetric part of J: closely related to stability properties
J.T - J

⎡0  0⎤
⎢    ⎥
⎣0  0⎦

In [ ]:
# all eigenvals have to have a real component for linearization, no eigenvals can be purely imaginary
# Hartmann-Gromann theorem
J.eigenvals()

⎧                                           __________________________________
⎨                         2  2    2  2     ╱         2                        
⎩-\lambdaₖ - \lambda_w - k ⋅y  - w ⋅y  - ╲╱  \lambdaₖ  - 2⋅\lambdaₖ⋅\lambda_w 

______________________________________________________________________________
              2  2               2  2            2                2  2        
- 2⋅\lambdaₖ⋅k ⋅y  + 2⋅\lambdaₖ⋅w ⋅y  + \lambda_w  + 2⋅\lambda_w⋅k ⋅y  - 2⋅\la

_________________________________________________________________________     
        2  2         2  2                3    4  4       2  2  4    4  4      
mbda_w⋅w ⋅y  + 4⋅\tau ⋅y  - 16⋅\tau⋅k⋅w⋅y  + k ⋅y  + 14⋅k ⋅w ⋅y  + w ⋅y  : 1, 

                                           ___________________________________
                         2  2    2  2     ╱         2                         
-\lambdaₖ - \lambda_w - k ⋅y  - w ⋅y  + ╲╱  \lambdaₖ  - 2⋅\lambdaₖ⋅\lambda_w -

________________________________________________

In [ ]:
# linearizing around fixed-point
H0 = H.subs(subs).subs(x0)
J0 = J.subs(subs).subs(x0)

In [ ]:
J0

⎡-2⋅\lambda   2⋅\tau⋅y ⎤
⎢                      ⎥
⎣ 2⋅\tau⋅y   -2⋅\lambda⎦

In [ ]:
J0.subs({a:0}).eigenvals()
# both in eigenvalues and fixed points -- underscore that this is also in stationary points
# only meaningful for sigma < tau^2
# for sigma > tau^2 --> linearize around 0, 0
# would have a different expression for 0, 0
# bifurcation -- when the stationary points approach the 0, 0 point --> pitchfork bifurcation

{-2⋅\lambda - 2⋅\tau⋅y: 1, -2⋅\lambda + 2⋅\tau⋅y: 1}

In [ ]:
# J0.subs({sig:sym.Rational(1,2)}).eigenvals()

In [ ]:
# gives us the bounds of sigma
# possibly: big tau makes the game easier (larger reaching task is easier)

#lambda_bar = max eigenvalue, upper bound rate of convergence in the neighborhood of fixed point

H0.eigenvals()

{2⋅\lambda - 2⋅\tau⋅y: 1, 2⋅\lambda + 2⋅\tau⋅y: 1}

When it comes to $J = Df(x_0)$, we care about whether **all eigenvalues have negative real part**,

$$ \forall \lambda \in \operatorname{spec} Df(x_0) : \operatorname{Real}\lambda < 0, $$

because we have the bound

$$ \| x(t) - x_0 \| \leq e^{\overline{\lambda} t} \| x(0) - x_0 \| $$

where

$$ \overline{\lambda} = \max\{\operatorname{Real}\lambda : \lambda\in \operatorname{spec} Df(x_0)\}. $$

Continuous-time gradient descent has dynamics:

$$ \dot{x} = - De(x) = f(x), $$

So on time horizon $\Delta > 0$, we have approximately

$$ x(t + \Delta) - x(t) \approx \Delta f(x(t)). $$

Discrete-time gradient descent has dynamics:

$$ x^+ = x - \Gamma De(x) = x + \Gamma f(x) = F(x), $$

Note that if $x_0$ is stationary:

$$ De(x_0) = 0 \implies f(x_0) = 0 \implies \dot{x}_0 = 0 $$

$$ De(x_0) = 0 \implies F(x_0) = x_0 $$

In [ ]:
Gamma = sym.Matrix.diag([gam_k,gam_w])
Gamma

⎡\gammaₖ     0    ⎤
⎢                 ⎥
⎣   0     \gamma_w⎦

In [ ]:
# Recall: f = -1*de = -1*[de_dk, de_dw]
f

⎡-2⋅\lambdaₖ⋅k + 2⋅w⋅y⋅(\tau - k⋅w⋅y) ⎤
⎢                                     ⎥
⎣-2⋅\lambda_w⋅w + 2⋅k⋅y⋅(\tau - k⋅w⋅y)⎦

In [ ]:
# Set F as the matrix to describe the update to k+, w+
F = sym.Matrix([k,w]) + Gamma * f
F

⎡ \gammaₖ⋅(-2⋅\lambdaₖ⋅k + 2⋅w⋅y⋅(\tau - k⋅w⋅y)) + k ⎤
⎢                                                    ⎥
⎣\gamma_w⋅(-2⋅\lambda_w⋅w + 2⋅k⋅y⋅(\tau - k⋅w⋅y)) + w⎦

In [ ]:
# # Set F as the matrix to describe the update to k+, w+
# F = sym.Matrix([k,w]) + Gamma * f.subs(subs)
# F

In [ ]:
# Discrete time, derivative of F
DF = F.jacobian([k,w])
DF

⎡           ⎛                 2  2⎞                  ⎛         2              
⎢   \gammaₖ⋅⎝-2⋅\lambdaₖ - 2⋅w ⋅y ⎠ + 1      \gammaₖ⋅⎝- 2⋅k⋅w⋅y  + 2⋅y⋅(\tau -
⎢                                                                             
⎢         ⎛         2                     ⎞             ⎛                  2  
⎣\gamma_w⋅⎝- 2⋅k⋅w⋅y  + 2⋅y⋅(\tau - k⋅w⋅y)⎠    \gamma_w⋅⎝-2⋅\lambda_w - 2⋅k ⋅y

       ⎞⎤
 k⋅w⋅y)⎠⎥
        ⎥
2⎞      ⎥
 ⎠ + 1  ⎦

In [ ]:
# x0 = {k:k0,w:w0}
DF.subs(x0).subs({lam_k:lam, lam_w:lam})

⎡-2⋅\gammaₖ⋅\lambda + 1     2⋅\gammaₖ⋅\tau⋅y    ⎤
⎢                                               ⎥
⎣  2⋅\gamma_w⋅\tau⋅y     -2⋅\gamma_w⋅\lambda + 1⎦

In [ ]:
# Sub in the stationary point for the eignenvalues
# Take Jacobian of F, find eigenvalues and evaluate at the fixed point
DF.eigenvals()

⎧                                                                             
⎨                             2  2                                  2  2     ╱
⎩-\gammaₖ⋅\lambdaₖ - \gammaₖ⋅w ⋅y  - \gamma_w⋅\lambda_w - \gamma_w⋅k ⋅y  - ╲╱ 

______________________________________________________________________________
        2         2            2           2  2          2  4  4              
 \gammaₖ ⋅\lambdaₖ  + 2⋅\gammaₖ ⋅\lambdaₖ⋅w ⋅y  + \gammaₖ ⋅w ⋅y  - 2⋅\gammaₖ⋅\

______________________________________________________________________________
                                                          2  2                
gamma_w⋅\lambdaₖ⋅\lambda_w - 2⋅\gammaₖ⋅\gamma_w⋅\lambdaₖ⋅k ⋅y  - 2⋅\gammaₖ⋅\ga

______________________________________________________________________________
                 2  2                          2  2                           
mma_w⋅\lambda_w⋅w ⋅y  + 4⋅\gammaₖ⋅\gamma_w⋅\tau ⋅y  - 16⋅\gammaₖ⋅\gamma_w⋅\tau

________________________________________________

In [ ]:
DF.subs(x0).subs({lam_k:lam, lam_w:lam}).eigenvals()

⎧                                         ____________________________________
⎨                                        ╱        2        2                  
⎩-\gammaₖ⋅\lambda - \gamma_w⋅\lambda - ╲╱  \gammaₖ ⋅\lambda  - 2⋅\gammaₖ⋅\gamm

________________________________________________________________              
           2                          2  2           2        2               
a_w⋅\lambda  + 4⋅\gammaₖ⋅\gamma_w⋅\tau ⋅y  + \gamma_w ⋅\lambda   + 1: 1, -\gam

                                    __________________________________________
                                   ╱        2        2                        
maₖ⋅\lambda - \gamma_w⋅\lambda + ╲╱  \gammaₖ ⋅\lambda  - 2⋅\gammaₖ⋅\gamma_w⋅\l

__________________________________________________________       ⎫
     2                          2  2           2        2        ⎬
ambda  + 4⋅\gammaₖ⋅\gamma_w⋅\tau ⋅y  + \gamma_w ⋅\lambda   + 1: 1⎭

In [ ]:
DF.subs(x0).subs({lam_k:lam, lam_w:lam}).subs({tau:1}).eigenvals()

⎧                                         ____________________________________
⎨                                        ╱        2        2                  
⎩-\gammaₖ⋅\lambda - \gamma_w⋅\lambda - ╲╱  \gammaₖ ⋅\lambda  - 2⋅\gammaₖ⋅\gamm

__________________________________________________________                    
           2                       2           2        2                     
a_w⋅\lambda  + 4⋅\gammaₖ⋅\gamma_w⋅y  + \gamma_w ⋅\lambda   + 1: 1, -\gammaₖ⋅\l

                              ________________________________________________
                             ╱        2        2                             2
ambda - \gamma_w⋅\lambda + ╲╱  \gammaₖ ⋅\lambda  - 2⋅\gammaₖ⋅\gamma_w⋅\lambda 

______________________________________________       ⎫
                       2           2        2        ⎬
 + 4⋅\gammaₖ⋅\gamma_w⋅y  + \gamma_w ⋅\lambda   + 1: 1⎭

In [ ]:
DF.subs(x0).subs({lam_k:lam, lam_w:lam}).subs({tau:1}).eigenvals()

⎧                                         ____________________________________
⎨                                        ╱        2        2                  
⎩-\gammaₖ⋅\lambda - \gamma_w⋅\lambda - ╲╱  \gammaₖ ⋅\lambda  - 2⋅\gammaₖ⋅\gamm

__________________________________________________________                    
           2                       2           2        2                     
a_w⋅\lambda  + 4⋅\gammaₖ⋅\gamma_w⋅y  + \gamma_w ⋅\lambda   + 1: 1, -\gammaₖ⋅\l

                              ________________________________________________
                             ╱        2        2                             2
ambda - \gamma_w⋅\lambda + ╲╱  \gammaₖ ⋅\lambda  - 2⋅\gammaₖ⋅\gamma_w⋅\lambda 

______________________________________________       ⎫
                       2           2        2        ⎬
 + 4⋅\gammaₖ⋅\gamma_w⋅y  + \gamma_w ⋅\lambda   + 1: 1⎭

In [ ]:
DF.subs(x0).subs({lam_k:lam, lam_w:lam}).subs({tau:1}).subs({lam:sym.Rational(1,1)}).eigenvals()

⎧                             ________________________________________________
⎨                            ╱        2  2                       2            
⎩-\gammaₖ⋅y - \gamma_w⋅y - ╲╱  \gammaₖ ⋅y  + 2⋅\gammaₖ⋅\gamma_w⋅y  - 16⋅\gamma

__________________________________________________                            
                                             2  2                             
ₖ⋅\gamma_w⋅y + 16⋅\gammaₖ⋅\gamma_w + \gamma_w ⋅y   + 1: 1, -\gammaₖ⋅y - \gamma

          ____________________________________________________________________
         ╱        2  2                       2                                
_w⋅y + ╲╱  \gammaₖ ⋅y  + 2⋅\gammaₖ⋅\gamma_w⋅y  - 16⋅\gammaₖ⋅\gamma_w⋅y + 16⋅\g

______________________________       ⎫
                         2  2        ⎬
ammaₖ⋅\gamma_w + \gamma_w ⋅y   + 1: 1⎭

In [ ]:
DF.subs(x0).subs({lam_k:lam, lam_w:lam}).subs({tau:1}).subs({lam:sym.Rational(1,1)}).subs({gam_k:1}).eigenvals()

⎧                     ________________________________________________________
⎨                    ╱         2  2               2                           
⎩-\gamma_w⋅y - y - ╲╱  \gamma_w ⋅y  + 2⋅\gamma_w⋅y  - 16⋅\gamma_w⋅y + 16⋅\gamm

_________                              _______________________________________
       2                              ╱         2  2               2          
a_w + y   + 1: 1, -\gamma_w⋅y - y + ╲╱  \gamma_w ⋅y  + 2⋅\gamma_w⋅y  - 16⋅\gam

__________________________       ⎫
                        2        ⎬
ma_w⋅y + 16⋅\gamma_w + y   + 1: 1⎭

In [ ]:
DF.subs(x0).subs({lam_k:lam, lam_w :lam}).subs({tau:1, y:sym.Rational(1,2)}).subs({lam:sym.Rational(1,1)}).subs({gam_k:1}).eigenvals()

⎧                _____________________________                         _______
⎪               ╱         2                                           ╱       
⎨  \gamma_w   ╲╱  \gamma_w  + 34⋅\gamma_w + 1    1       \gamma_w   ╲╱  \gamma
⎪- ──────── - ──────────────────────────────── + ─: 1, - ──────── + ──────────
⎩     2                      2                   2          2                 

______________________       ⎫
  2                          ⎪
_w  + 34⋅\gamma_w + 1    1   ⎬
────────────────────── + ─: 1⎪
     2                   2   ⎭

In [ ]:
DF.subs(x0).subs({lam_k:lam, lam_w :lam}).subs({tau:1, y:sym.Rational(0.5)}).subs({lam:sym.Rational(1,1)}).subs({gam_k:1, gam_w:1}).eigenvals()

{-3: 1, 3: 1}

When it comes to $DF(x_0)$, we care about whether **all** eigenvalues have magnitude smaller than $1$,

$$ \forall \lambda \in \operatorname{spec} DF(x_0) : |\lambda| < 1, $$

because we have the bound

$$ \| x(k) - x_0 \| \leq \widetilde{\lambda}^k \| x(0) - x_0 \| $$

where

$$ \widetilde{\lambda} = \max\{|\lambda| : \lambda\in \operatorname{spec} DF(x_0)\}. $$

In [ ]:
#x0 = k0, w0
DF.subs({lam_k:lam, lam_w:lam}).subs({a:0, tau:1}).eigenvals()

⎧                                                                          ___
⎨                            2  2                                2  2     ╱   
⎩-\gammaₖ⋅\lambda - \gammaₖ⋅w ⋅y  - \gamma_w⋅\lambda - \gamma_w⋅k ⋅y  - ╲╱  \g

______________________________________________________________________________
     2        2            2          2  2          2  4  4                   
ammaₖ ⋅\lambda  + 2⋅\gammaₖ ⋅\lambda⋅w ⋅y  + \gammaₖ ⋅w ⋅y  - 2⋅\gammaₖ⋅\gamma

______________________________________________________________________________
          2                               2  2                               2
_w⋅\lambda  - 2⋅\gammaₖ⋅\gamma_w⋅\lambda⋅k ⋅y  - 2⋅\gammaₖ⋅\gamma_w⋅\lambda⋅w 

______________________________________________________________________________
  2                        2  2  4                            3               
⋅y  + 14⋅\gammaₖ⋅\gamma_w⋅k ⋅w ⋅y  - 16⋅\gammaₖ⋅\gamma_w⋅k⋅w⋅y  + 4⋅\gammaₖ⋅\g

________________________________________________

In [ ]:
# iterative gradient descent converges?
# Two-Learner paper connection: learning rates can't be too large or small
# DF.subs(x0).subs({a:0, tau:1, lam:sym.Rational(1,2)}).eigenvals()
DF.subs(x0).subs({a:0, tau:1, lam_k:lam, lam_w:lam}).eigenvals()

⎧                             ________________________________________________
⎨                            ╱        2  2                              2     
⎩-\gammaₖ⋅y - \gamma_w⋅y - ╲╱  \gammaₖ ⋅y  + 16⋅\gammaₖ⋅\gamma_w⋅\lambda  - 16

___________________________________________________________________           
                                                  2           2  2            
⋅\gammaₖ⋅\gamma_w⋅\lambda⋅y + 2⋅\gammaₖ⋅\gamma_w⋅y  + \gamma_w ⋅y   + 1: 1, -\

                           ___________________________________________________
                          ╱        2  2                              2        
gammaₖ⋅y - \gamma_w⋅y + ╲╱  \gammaₖ ⋅y  + 16⋅\gammaₖ⋅\gamma_w⋅\lambda  - 16⋅\g

________________________________________________________________       ⎫
                                               2           2  2        ⎬
ammaₖ⋅\gamma_w⋅\lambda⋅y + 2⋅\gammaₖ⋅\gamma_w⋅y  + \gamma_w ⋅y   + 1: 1⎭

In [ ]:
## if lambda = 0
DF.subs(x0).subs({a:0, lam_k: lam, lam_w:lam}).subs({lam:sym.Rational(0)}).eigenvals()

{1: 1, -2⋅\gammaₖ⋅\tau⋅y - 2⋅\gamma_w⋅\tau⋅y + 1: 1}

In [ ]:
# # substitution of sigma = 1/2

# DF.subs({a:0, tau:1, lam_k:lam, lam_w:lam, lam:sym.Rational(1,2)}).eigenvals()
DF.subs(x0).subs({a:0, tau:1, lam_k: lam, lam_w:lam}). subs({lam:sym.Rational(1,2)}).eigenvals()

⎧                             ________________________________________________
⎨                            ╱        2  2                       2            
⎩-\gammaₖ⋅y - \gamma_w⋅y - ╲╱  \gammaₖ ⋅y  + 2⋅\gammaₖ⋅\gamma_w⋅y  - 8⋅\gammaₖ

________________________________________________                              
                                           2  2                               
⋅\gamma_w⋅y + 4⋅\gammaₖ⋅\gamma_w + \gamma_w ⋅y   + 1: 1, -\gammaₖ⋅y - \gamma_w

        ______________________________________________________________________
       ╱        2  2                       2                                  
⋅y + ╲╱  \gammaₖ ⋅y  + 2⋅\gammaₖ⋅\gamma_w⋅y  - 8⋅\gammaₖ⋅\gamma_w⋅y + 4⋅\gamma

__________________________       ⎫
                     2  2        ⎬
ₖ⋅\gamma_w + \gamma_w ⋅y   + 1: 1⎭

In [ ]:
# the case when y = tau = 1
# lambda_brain = lamdba_decoder = 1/2
DF.subs(x0).subs({a:0, tau:1, lam_k: lam, lam_w:lam, y:1}). subs({lam:sym.Rational(1,2)}).eigenvals()

{1 - 2⋅\gammaₖ: 1, 1 - 2⋅\gamma_w: 1}

In [ ]:
DF.subs(x0).subs({lam:sym.Rational(1,2), gam_k:sym.Rational(1), gam_w:sym.Rational(1), tau:1, a:0}).eigenvals()


⎧                                 ____________________________________________
⎨                                ╱         2                                  
⎩-\lambdaₖ - \lambda_w - 2⋅y - ╲╱  \lambdaₖ  - 2⋅\lambdaₖ⋅\lambda_w + \lambda_

____________________                                          ________________
 2      2                                                    ╱         2      
w  + 4⋅y  - 8⋅y + 4  + 2: 1, -\lambdaₖ - \lambda_w - 2⋅y + ╲╱  \lambdaₖ  - 2⋅\

________________________________________________       ⎫
                             2      2                  ⎬
lambdaₖ⋅\lambda_w + \lambda_w  + 4⋅y  - 8⋅y + 4  + 2: 1⎭

In [ ]:
DF.subs(x0).subs({a:0, tau:1}).eigenvals()

⎧                                                                             
⎨                                                                             
⎩\gammaₖ⋅\lambda - \gammaₖ⋅\lambdaₖ - \gammaₖ⋅y + \gamma_w⋅\lambda - \gamma_w⋅

                            __________________________________________________
                           ╱        2        2            2                   
\lambda_w - \gamma_w⋅y - ╲╱  \gammaₖ ⋅\lambda  - 2⋅\gammaₖ ⋅\lambda⋅\lambdaₖ -

______________________________________________________________________________
          2                    2         2            2                     2 
 2⋅\gammaₖ ⋅\lambda⋅y + \gammaₖ ⋅\lambdaₖ  + 2⋅\gammaₖ ⋅\lambdaₖ⋅y + \gammaₖ ⋅

______________________________________________________________________________
 2                              2                                             
y  + 14⋅\gammaₖ⋅\gamma_w⋅\lambda  + 2⋅\gammaₖ⋅\gamma_w⋅\lambda⋅\lambdaₖ + 2⋅\g

________________________________________________

In [ ]:
DF.subs(x0).subs({a:0}).eigenvals()

⎧                                                                             
⎨                                                                             
⎩\gammaₖ⋅\lambda - \gammaₖ⋅\lambdaₖ - \gammaₖ⋅\tau⋅y + \gamma_w⋅\lambda - \gam

                                      ________________________________________
                                     ╱        2        2            2         
ma_w⋅\lambda_w - \gamma_w⋅\tau⋅y - ╲╱  \gammaₖ ⋅\lambda  - 2⋅\gammaₖ ⋅\lambda⋅

______________________________________________________________________________
                    2                         2         2            2        
\lambdaₖ - 2⋅\gammaₖ ⋅\lambda⋅\tau⋅y + \gammaₖ ⋅\lambdaₖ  + 2⋅\gammaₖ ⋅\lambda

______________________________________________________________________________
                  2     2  2                              2                   
ₖ⋅\tau⋅y + \gammaₖ ⋅\tau ⋅y  + 14⋅\gammaₖ⋅\gamma_w⋅\lambda  + 2⋅\gammaₖ⋅\gamma

________________________________________________

In [ ]:
# As in the block of text above, the eigenvalue that has the larger value is the one to pay attention to
# Take the log of ||KW - K_oW_o|| closer to the stationary point
# Compare that with the t*log(eigenvalue) to get the convergence rate
DF.subs(x0).subs({lam:sym.Rational(1,2), tau:1}).eigenvals()

# eigenvalues have to be within the unit circle when sigma = 1/2

⎧                                                                             
⎪                                                                             
⎨                                \gammaₖ                                     \
⎪-\gammaₖ⋅\lambdaₖ - \gammaₖ⋅y + ─────── - \gamma_w⋅\lambda_w - \gamma_w⋅y + ─
⎩                                   2                                         

             _________________________________________________________________
            ╱          2         2            2                       2       
gamma_w   ╲╱  4⋅\gammaₖ ⋅\lambdaₖ  + 8⋅\gammaₖ ⋅\lambdaₖ⋅y - 4⋅\gammaₖ ⋅\lambd
─────── - ────────────────────────────────────────────────────────────────────
  2                                                                           

______________________________________________________________________________
              2  2            2            2                                  
aₖ + 4⋅\gammaₖ ⋅y  - 4⋅\gammaₖ ⋅y + \gammaₖ  - 8⋅\

In [ ]:
# F_num = sym.lambdify([k,w],F.subs({sig:sym.Rational(1,2),gam_k:2.001,gam_w:.9}))
# x = [np.random.randn(2)]
# for i in range(100):
#   x.append(F_num(*x[-1]).flatten())
# x = np.asarray(x)
# plt.plot(x)